# Running HMMER and getting dbCAN output.

While you can get dbCAN predictions for a proteome (or predicted proteome) using their web interface (<http://csbl.bmb.uga.edu/dbCAN/annotate.php>), it might be easier to run the search locally if you have a lot of sequences or multiple proteomes to process.

Here I go through this process of getting dbCAN formatted output in preparation for the training of CATAStrophy.
Note that although I am using a jupyter notebook, the actual code is run only in bash `%%magic` cells meaning that you could just copy-paste into your terminal.

To rerun these steps you'll need to be running a UNIX-style command line (this was performed in Fedora 25, but it should be fine in Linux/MacOS), and you'll need [HMMER](http://hmmer.org/) installed (we used v3.1b2) and some version of perl (included in most linux distros; this is used in dbCANs output parser script).

In [1]:
import datetime

# TODAY = "20180324" 
TODAY = datetime.datetime.utcnow().strftime("%Y%m%d")

## Preparing dbCAN

First we need to download and prepare the dbCAN HMMs.
We'll also need to download the parser script.

In [2]:
%%bash
# Make sure that you are working in a directory that you can safely work in!

pwd

/home/darcyabjones/Libs/catastrophy/notebooks


In [3]:
%%bash
mkdir -p ./data

# Comment out any versions that you don't want
# Here we download multiple versions to develop models for each version.
wget -qc -P ./data http://csbl.bmb.uga.edu/dbCAN/download/dbCAN-fam-HMMs.txt.v4
wget -qc -P ./data http://csbl.bmb.uga.edu/dbCAN/download/dbCAN-fam-HMMs.txt.v5
wget -qc -P ./data http://csbl.bmb.uga.edu/dbCAN/download/dbCAN-fam-HMMs.txt.v6
wget -qc -P ./data http://cys.bios.niu.edu/dbCAN2/download/Databases/dbCAN-HMMdb-V7.txt

wget -qc -P ./data http://csbl.bmb.uga.edu/dbCAN/download/hmmscan-parser.sh

The `-q` flag to wget just suppresses the output of wget so that it doesn't clog up the notebook.
The files are now in the 'data' folder.

In [4]:
ls ./data

dbCAN-fam-HMMs.txt.v4  dbCAN-fam-HMMs.txt.v6  hmmscan-parser.sh
dbCAN-fam-HMMs.txt.v5  dbCAN-HMMdb-V7.txt


Notice that we downloaded multiple specific versions of the database.
CATAStrophy won't necessarily work correctly with future versions of the database unless we retrain the models.

Next we need to create a HMMER formatted database from the .txt file we downloaded.

In [5]:
%%bash
# the rm is just here so I can run the notebook without errors
rm -f ./data/dbCAN-fam-HMMs.txt.v4.*
hmmpress ./data/dbCAN-fam-HMMs.txt.v4

rm -f ./data/dbCAN-fam-HMMs.txt.v5.*
hmmpress ./data/dbCAN-fam-HMMs.txt.v5

rm -f ./data/dbCAN-fam-HMMs.txt.v6.*
hmmpress ./data/dbCAN-fam-HMMs.txt.v6

rm -f ./data/dbCAN-HMMdb-V7.txt.*
hmmpress ./data/dbCAN-HMMdb-V7.txt

Working...    done.
Pressed and indexed 345 HMMs (345 names).
Models pressed into binary file:   ./data/dbCAN-fam-HMMs.txt.v4.h3m
SSI index for binary model file:   ./data/dbCAN-fam-HMMs.txt.v4.h3i
Profiles (MSV part) pressed into:  ./data/dbCAN-fam-HMMs.txt.v4.h3f
Profiles (remainder) pressed into: ./data/dbCAN-fam-HMMs.txt.v4.h3p
Working...    done.
Pressed and indexed 360 HMMs (360 names and 1 accessions).
Models pressed into binary file:   ./data/dbCAN-fam-HMMs.txt.v5.h3m
SSI index for binary model file:   ./data/dbCAN-fam-HMMs.txt.v5.h3i
Profiles (MSV part) pressed into:  ./data/dbCAN-fam-HMMs.txt.v5.h3f
Profiles (remainder) pressed into: ./data/dbCAN-fam-HMMs.txt.v5.h3p
Working...    done.
Pressed and indexed 585 HMMs (585 names and 1 accessions).
Models pressed into binary file:   ./data/dbCAN-fam-HMMs.txt.v6.h3m
SSI index for binary model file:   ./data/dbCAN-fam-HMMs.txt.v6.h3i
Profiles (MSV part) pressed into:  ./data/dbCAN-fam-HMMs.txt.v6.h3f
Profiles (remainder) pressed int

## Running HMMER

Now that the database is ready, we can process our sequences.
This is pretty easily done with the command:

```bash
hmmscan --domtblout my_seqs_hmmer.csv ./data/dbCAN-fam-HMMs.txt my_seqs.fasta > my_seqs_hmmer.txt
```

The native dbCAN script uses the "domain table" (provided by `--domtblout`) as input, which is a much more friendly space-delimited format.
The txt output is the raw hmmer output. It is a lot more information rich but also harder to parse.

The HMMER output formats should be more "stable" than the dbCAN format (which recently changed and hasn't been updated on their webserver).
Ideally we want to support all three formats, but focussing on the HMMER ones.

Because in our training we need to use lots of proteomes I'll be running this command using gnu parallel <https://www.gnu.org/software/parallel/>.
Frankly, the documentation is confusion for this but the tutorial is fairly usable <https://www.gnu.org/software/parallel/parallel_tutorial.html>.
They're also super aggressive with their citation reminders!

Anyway, install with your favourite package manager.
In the command we pipe a list of fasta files to parallel.
It runs the provided command for each input line, substituting `{}` with the input filename, and `{/.}` with the input filename _sans_ extension or path.
The final line 

In [7]:
%%bash
# Use all available CPUs by default
# Replace number if you want to restrict CPU usage.
NCPU=$(grep -c "processor" /proc/cpuinfo)

# Replace this with the directory that your fastas are in.
FASTA_DIR="20170531-trophic_prediction_fastas/fastas"

# Remove versions from this list as desired
VERSIONS="v4 v5 v6 v7"

# Because v7 has different naming convention, use
# associative array for mapping.
declare -A DBPATHS
DBPATHS[v4]=data/dbCAN-fam-HMMs.txt.v4
DBPATHS[v5]=data/dbCAN-fam-HMMs.txt.v5
DBPATHS[v6]=data/dbCAN-fam-HMMs.txt.v6
DBPATHS[v7]=data/dbCAN-HMMdb-V7.txt

for v in ${VERSIONS}; do
    mkdir -p "01-run_hmms/${v}"
    find "${FASTA_DIR}" -type f \
        | parallel -j ${NCPU} "hmmscan --domtblout 01-run_hmms/${v}/{/.}_hmmer.csv ${DBPATHS[${v}]} {} > 01-run_hmms/${v}/{/.}_hmmer.txt" \
        && echo "${v} is done"
done

v4 is done
v5 is done
v6 is done
v7 is done


So that's done, now we need to get the special dbCAN formatted output.

Note that if you are just running CATAStrophy, you can just use the domain table as input.

Essentially this script just filters rows by some criteria (if alignment > 80aa, use E-value < 1e-5, otherwise use E-value < 1e-3; covered fraction of HMM > 0.3) and outputs a subset of the columns.

In [8]:
%%bash

# Remove versions from this list as desired
VERSIONS="v4 v5 v6 v7"

for v in ${VERSIONS}; do
    for f in $(ls 01-run_hmms/${v}/*_hmmer.csv | grep -v "dbcan"); do
        f_noext="${f%.*}"
        bash ./data/hmmscan-parser.sh "${f}" > "${f_noext}_dbcan.csv.tmp" \
        && mv "${f_noext}_dbcan.csv.tmp" "${f_noext}_dbcan.csv"
    done
done

In [9]:
ls 01-run_hmms/v5/ | head

Abisporus_varbisporusH97.v2.FilteredModels3.proteins_hmmer.csv
Abisporus_varbisporusH97.v2.FilteredModels3.proteins_hmmer_dbcan.csv
Abisporus_varbisporusH97.v2.FilteredModels3.proteins_hmmer.txt
Abisporus_varburnettii.v2.FilteredModels1.proteins_hmmer.csv
Abisporus_varburnettii.v2.FilteredModels1.proteins_hmmer_dbcan.csv
Abisporus_varburnettii.v2.FilteredModels1.proteins_hmmer.txt
Albugo_laibachii.ENA1.27.pep.all_hmmer.csv
Albugo_laibachii.ENA1.27.pep.all_hmmer_dbcan.csv
Albugo_laibachii.ENA1.27.pep.all_hmmer.txt
Alternaria_alternata.Altal1.strainSRC1lrK2f.pep.all_hmmer.csv
ls: write error


In [10]:
!head 01-run_hmms/v5/Albugo_laibachii.ENA1.27.pep.all_hmmer.csv

#                                                                            --- full sequence --- -------------- this domain -------------   hmm coord   ali coord   env coord
# target name        accession   tlen query name           accession   qlen   E-value  score  bias   #  of  c-Evalue  i-Evalue  score  bias  from    to  from    to  from    to  acc description of target
#------------------- ---------- ----- -------------------- ---------- ----- --------- ------ ----- --- --- --------- --------- ------ ----- ----- ----- ----- ----- ----- ----- ---- ---------------------
GH129.hmm            -            618 CCA21570             -            611   0.00079   12.3   0.0   1   1   3.4e-06    0.0012   11.7   0.0   490   571   345   423   339   430 0.88 -
GH80.hmm             -             63 CCA21581             -           1335    0.0024   12.3   0.1   1   2      0.15        53   -1.6   0.0    30    48   237   255   234   258 0.89 -
GH80.hmm             -             63 CCA21581      

In [11]:
!head 01-run_hmms/v5/Albugo_laibachii.ENA1.27.pep.all_hmmer.txt

# hmmscan :: search sequence(s) against a profile database
# HMMER 3.2.1 (June 2018); http://hmmer.org/
# Copyright (C) 2018 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query sequence file:             20170531-trophic_prediction_fastas/fastas/Albugo_laibachii.ENA1.27.pep.all.fasta
# target HMM database:             data/dbCAN-fam-HMMs.txt.v5
# per-dom hits tabular output:     01-run_hmms/v5/Albugo_laibachii.ENA1.27.pep.all_hmmer.csv
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -



In [12]:
!head 01-run_hmms/v5/Albugo_laibachii.ENA1.27.pep.all_hmmer_dbcan.csv

GH38.hmm	269	CCA13903	1025	3.9e-75	1	269	48	349	0.996282527881041
CBM47.hmm	128	CCA13913	1044	2.3e-10	6	118	764	877	0.875
GT4.hmm	160	CCA13921	458	7.2e-06	34	157	263	383	0.76875
GH5.hmm	275	CCA13966	770	1.8e-29	3	239	59	574	0.858181818181818
CE2.hmm	209	CCA14218	469	5.6e-65	1	209	201	430	0.995215311004785
GH17.hmm	311	CCA14249	419	4.5e-19	29	306	133	374	0.890675241157556
CBM32.hmm	124	CCA14262	583	4.1e-09	14	109	464	563	0.766129032258065
GT31.hmm	192	CCA14287	372	4.9e-16	50	183	111	257	0.692708333333333
GH30.hmm	417	CCA14291	682	2.9e-106	4	374	108	506	0.887290167865707
GH72.hmm	312	CCA14303	556	7.5e-73	6	284	58	372	0.891025641025641


## Getting the HMM lengths

because the HMMER plain text output doesn't contain the HMM length anywhere, I'll need to save them.

In [16]:
import re
import json
from collections import defaultdict

from catas.parsers import split_hmm

In [17]:
regex = re.compile(r"\s+")
output = defaultdict(dict)

VERSIONS = {
    "v4": "data/dbCAN-fam-HMMs.txt.v4",
    "v5": "data/dbCAN-fam-HMMs.txt.v5",
    "v6": "data/dbCAN-fam-HMMs.txt.v6",
    "v7": "data/dbCAN-HMMdb-V7.txt",
}
    
for version, db in VERSIONS.items():
    with open(db) as handle:
        current_name = None
        for line in handle:
            if line.startswith("NAME"):
                line = regex.split(line)
                current_name = split_hmm(line[1])
            elif line.startswith("LENG"):
                line = regex.split(line, maxsplit=1)
                if current_name is None:
                    print(line)
                    raise

                output[version][current_name] = int(line[1])
                current_name = None

And we'll save that as a JSON file in the catas data directory.

In [19]:
for version in VERSIONS:
    with open("../catas/data/{}-{}-hmm_lengths.json".format(version, TODAY), "w") as handle:
        json.dump(output[version], handle)

## Preparing test data

In [20]:
import subprocess
from subprocess import Popen
from subprocess import PIPE

import catas.data

fasta_file = catas.data.sample_fasta()

In [21]:
for version, db in VERSIONS.items():
    command = [
        "hmmscan",
        "--domtblout",
        "../catas/data/{}-{}-test_hmmer.csv".format(version, TODAY),
        db,
        fasta_file
        ]
    call = Popen(command, stdout=PIPE)
    stdout, stderr = call.communicate()

    with open("../catas/data/{}-{}-test_hmmer.txt".format(version, TODAY), "wb") as handle:
        handle.write(stdout)

In [22]:
for version in VERSIONS:
    command = [
        "bash",
        "./data/hmmscan-parser.sh",
        "../catas/data/{}-{}-test_hmmer.csv".format(version, TODAY)
        ]
    call = Popen(command, stdout=PIPE)
    stdout, stderr = call.communicate()
    
    with open("../catas/data/{}-{}-test_dbcan.csv".format(version, TODAY), "wb") as handle:
        handle.write(stdout)